In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
import collections
from itertools import combinations

In [3]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1866,tt0160797,60000000,71000000,Rules of Engagement,Tommy Lee Jones|Samuel L. Jackson|Guy Pearce|B...,William Friedkin,A hero should never have to stand alone.,"Part war movie, part courtroom thriller, this ...",128,Action|Adventure|Drama|History|War,Paramount Pictures|Seven Arts Pictures|Munich ...,4/7/2000,5.8,2000
1749,tt0462244,6000000,18197398,Daddy Day Camp,Cuba Gooding Jr.|Tamala Jones|Paul Rae|Lochlyn...,Fred Savage,The summer is going to be in tents.,Seeking to offer his son the satisfying summer...,89,Comedy|Family,Revolution Studios|TriStar Pictures,8/8/2007,4.8,2007
22,tt2126355,110000000,470490832,San Andreas,Dwayne Johnson|Alexandra Daddario|Carla Gugino...,Brad Peyton,"A rescue pilot survived an earthquake, this is...",In the aftermath of a massive earthquake in Ca...,114,Action|Drama|Thriller,New Line Cinema|Village Roadshow Pictures|Warn...,5/27/2015,6.1,2015
1292,tt1322269,25000000,74188937,August: Osage County,Meryl Streep|Julia Roberts|Chris Cooper|Ewan M...,John Wells,Misery loves family.,A look at the lives of the strong-willed women...,121,Comedy|Drama,Jean Doumanian Productions|Smokehouse Pictures,12/26/2013,6.8,2013
586,tt0212985,87000000,351692268,Hannibal,Anthony Hopkins|Julianne Moore|Gary Oldman|Ray...,Ridley Scott,"His genius undeniable. His evil, unspeakable.",After having successfully eluded the authoriti...,131,Crime|Drama|Thriller|Horror,Universal Pictures|Scott Free Productions|Metr...,2/8/2001,6.3,2001


In [4]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Preprocessing


In [5]:
#crate a dictionary for answers

answers = {} 


#the profit is not represented in data, so we need to add column with profit 
#profit = revenue - budget

data['profit'] = data['revenue'] - data['budget']

#The new column with a length of movies titles

data['title_len'] = data['original_title'].map(lambda s: len(s))

#the new column with a length of their overview

data['overview_len'] = data['overview'].map(lambda s: len(s.split()))

#Some columns consist symbol "|".
#for further analysis we need to add a new column with separated parameters
#for counting every element in the list we will use explode function

#add new directors list column

data['dir_list'] = data.director.str.split('|')

#add new genres  list column

data['genres_list'] = data.genres.str.split('|')

#add new column with a cast list

data['cast_list'] = data.cast.str.split('|')

#Add new column with a companies list

data['companies_list'] = data.production_companies.str.split('|')

#split date in the release date

data['month'] = [int(x.split('/')[0]) for x in data['release_date']]



# 1.  Which movie has biggest budget?

In [6]:
#variant1

max_budget = data[['imdb_id','budget','original_title']].sort_values(['budget'],ascending=False).head(1)
print(max_budget)

answers['1'] = 'Pirates of the Caribbean: On Stranger Tides (tt1298650)'

       imdb_id     budget                               original_title
723  tt1298650  380000000  Pirates of the Caribbean: On Stranger Tides


In [7]:
#variant2

max_budget2 = data[data.budget == data.budget.max()].original_title
print(max_budget2)

723    Pirates of the Caribbean: On Stranger Tides
Name: original_title, dtype: object


# 2. Which movie is the longest (in minutes)?

In [8]:
longest_movie = data[['imdb_id','runtime','original_title']].sort_values(['runtime'],ascending=False).head(1)
print(longest_movie)

answers['2'] = 'Gods and Generals (tt0279111)'

        imdb_id  runtime     original_title
1157  tt0279111      214  Gods and Generals


# 3. Which movie is the shortest (in minutes)?





In [9]:
shortest_movie = data[['imdb_id','runtime','original_title']].sort_values(['runtime'],ascending=True).head(1) 
print(shortest_movie)

answers['3'] = 'Winnie the Pooh (tt1449283)'

       imdb_id  runtime   original_title
768  tt1449283       63  Winnie the Pooh


# 4. What is the average duration of the movie?


In [10]:
mean_runtime = round(data.runtime.mean())   
print(mean_runtime)

answers['4'] = '110'

110


# 5. What is median number of the movies duration? 

In [11]:
median_runtime = data.runtime.median() 
print(int(median_runtime))

answers['5'] = '107'

107


# 6.What movie is the most profitable
#### (profit = revenue - budget) 

In [12]:
#use the profit column  
profitable_movie = data[['imdb_id','original_title','profit', 'budget', 'revenue']].sort_values(['profit'],ascending=False).head(1) 
print(profitable_movie)

answers['6'] = 'Avatar (tt0499549)'

       imdb_id original_title      profit     budget     revenue
239  tt0499549         Avatar  2544505847  237000000  2781505847


# 7. What movie is the most unprofitable?

In [13]:
unprofitable_movie = data[['imdb_id','original_title','profit', 'budget', 'revenue']].sort_values(['profit'],ascending=True).head(1) 
print(unprofitable_movie)

answers['7'] = 'The Lone Ranger (tt1210819)'

        imdb_id   original_title     profit     budget   revenue
1245  tt1210819  The Lone Ranger -165710090  255000000  89289910


# 8. How much movies have profit more than budget?

In [14]:
profits_number = data[data.profit >0]
print(len(profits_number))

answers['8'] = '1478'

1478


# 9. Which movie was the most grossing in 2008?

In [15]:
#variant 1
year_2008 = data[data.release_year == 2008]  
highest_grossing = year_2008[year_2008.revenue == year_2008.revenue.max()].original_title
print(highest_grossing)

answers['9'] = 'The Dark Knight (tt0468569)'

599    The Dark Knight
Name: original_title, dtype: object


In [16]:
#variant2
highest_grossing2 = data[data.release_year == 2008].sort_values(['revenue'],ascending=False).head(1) 
highest_grossing2

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,...,vote_average,release_year,profit,title_len,overview_len,dir_list,genres_list,cast_list,companies_list,month
599,tt0468569,185000000,1001921825,The Dark Knight,Christian Bale|Michael Caine|Heath Ledger|Aaro...,Christopher Nolan,Why So Serious?,Batman raises the stakes in his war on crime. ...,152,Drama|Action|Crime|Thriller,...,8.1,2008,816921825,15,67,[Christopher Nolan],"[Drama, Action, Crime, Thriller]","[Christian Bale, Michael Caine, Heath Ledger, ...","[DC Comics, Legendary Pictures, Warner Bros., ...",7


# 10. The most unprofitable movies  in 2012 - 2014 (including)?


In [17]:
#variant1

years2012_2014 = data[data.release_year.isin([2012,2013,2014])]
unprofitable2012_2104 = years2012_2014[years2012_2014.profit == years2012_2014.profit.min()].original_title
print(unprofitable2012_2104)

answers['10'] = 'The Lone Ranger (tt1210819)'

1245    The Lone Ranger
Name: original_title, dtype: object


In [18]:
#variant2

years12_14 = data.query('2012<=release_year<=2014')
unprofitable12_14 = years12_14[years12_14.profit == years12_14.profit.min()].original_title
print(unprofitable12_14)

1245    The Lone Ranger
Name: original_title, dtype: object


# 11. Which genre is the most?

In [19]:
#variant1

exploded_genres = data.explode('genres_list')
common_genres = exploded_genres['genres_list'].value_counts().head(1)
print(common_genres)

answers['11'] = 'Drama'

Drama    782
Name: genres_list, dtype: int64


Variant 2

In [20]:
print(pd.Series(data['genres'].str.cat(sep='|').split('|')).value_counts().head(1))

Drama    782
dtype: int64


# 12. Movies with which genres are the most profitable?

In [21]:
#take dataframeprofits_number from task 8 and use on it code from task 11 

print(pd.Series(profits_number['genres'].str.cat(sep='|').split('|')).value_counts().head(1))

answers['12'] = 'Drama'

Drama    560
dtype: int64


# 13. Which director has the highest total revenue?

In [22]:
grossing_director = data.groupby(['director'])[['revenue']].sum().sort_values(['revenue'],ascending=False).head(1)
print(grossing_director)

answers['13'] = 'Peter Jackson'

                  revenue
director                 
Peter Jackson  6490593685


# 14. Which director made more movies in genre Action?

In [23]:
action_movies = data[data.genres.str.contains ("Action", na=False)]
dir_exploded = action_movies.explode('dir_list')
action_director = dir_exploded.dir_list.value_counts().head(1)
print(action_director)

answers['14'] = 'Robert Rodriguez'

Robert Rodriguez    9
Name: dir_list, dtype: int64


# 15. Movies with which actor had the highest revenue in 2012?


In [24]:
year_2012 = data[data.release_year == 2012]
exploded_actor = year_2012.explode('cast_list')
grossing_actor = exploded_actor.groupby(['cast_list'])[['revenue']].sum().sort_values(['revenue'],ascending=False).head(1)
print(grossing_actor)


answers['15'] = 'Chris Hemsworth'

                    revenue
cast_list                  
Chris Hemsworth  2027450773


# 16.  Which actor has starred in more big-budget films?

In [25]:
budget_movie = data[data.budget > data.budget.mean()]
exploded_actor2 = budget_movie.explode('cast_list')
grossing_actor2 = exploded_actor2.cast_list.value_counts().head(1)
print(grossing_actor2)

answers['16'] = 'Matt Damon'

Matt Damon    18
Name: cast_list, dtype: int64


# 17. What genre has Nicolas Cage done the most? 

In [26]:
Cage = data[data.cast.str.contains ("Nicolas Cage", na=False)]
print(pd.Series(Cage['genres'].str.cat(sep='|').split('|')).value_counts().head(1))

answers['17'] = 'Action'

Action    17
dtype: int64


# 18. The most unprofitable film from Paramount Pictures?

In [27]:
Paramount = data[data.production_companies.str.contains ("Paramount Pictures", na=False)]
unprofitable_Paramount = Paramount[['imdb_id','original_title','profit']].sort_values(['profit'],ascending=True).head(1) 
print(unprofitable_Paramount)

answers['18'] = 'K-19: The Widowmaker (tt0267626)'

       imdb_id        original_title    profit
925  tt0267626  K-19: The Widowmaker -64831034


# 19. What year was the most successful by the total box office? 

In [28]:
best_year = data.groupby(['release_year'])[['revenue']].sum().sort_values(['revenue'],ascending=False).head(1)
print(best_year)

answers['19'] = '2015'

                  revenue
release_year             
2015          25449202382


# 20. What's the most profitable year for Warner Bros. Studios? 

In [29]:
Warner = data[data.production_companies.str.contains ("Warner Bros", na=False)]
best_Warner = Warner.groupby(['release_year'])[['revenue']].sum().sort_values(['revenue'],ascending=False).head(1) 
print(best_Warner)

answers['20'] = '2014'

                 revenue
release_year            
2014          3243064519


# 21. In which month for all the years the most films were released in total? 

In [30]:
pivot_months = data.pivot_table(values='imdb_id',
                           index='release_year',
                           columns='month',
                           fill_value=0,
                           aggfunc='count')
display(pivot_months.sum().sort_values(ascending=False).head(1))

answers['21'] = '9'

month
9    227
dtype: int64

# 22. How much movies were released in summer? (June. July and August)

In [31]:
display(pivot_months.sum()[5:8].sum())

answers['22'] = '450'

450

# 23. For which director winter is the most productive? 

In [32]:
winter = data[(data.month.isin([1,2,12]))] 
winter_dir = winter.director.str.split('|').sum() 
display(Counter(winter_dir).most_common(1))

answers['23'] = 'Peter Jackson'


[('Peter Jackson', 7)]

# 24. Which studios makes the longest titles for their movies by symbols?

In [33]:
exploded_companies = data.explode('companies_list')
title_names = exploded_companies.groupby(['companies_list'])['title_len'].mean().sort_values(ascending=False)
print(title_names.head(1))

answers['24'] = 'Four By Two Productions'

companies_list
Four By Two Productions    83.0
Name: title_len, dtype: float64


# 25. Which studios makes the longest average description for their movies by words?

In [34]:
longest_overview = exploded_companies.groupby(['companies_list'])['overview_len'].mean().sort_values(ascending=False)
print(longest_overview.head(1))

answers['25']= 'Midnight Picture Show'

companies_list
Midnight Picture Show    175.0
Name: overview_len, dtype: float64


# 26. What movies are the top 1% in the list by vote_average?

In [35]:
top = data[data['vote_average'] > data['vote_average'].quantile(0.99)][['original_title','vote_average']]
print(top)

answers['26'] = 'Inside Out, The Dark Knight, 12 Years a Slave'

                                     original_title  vote_average
9                                        Inside Out           8.0
34                                             Room           8.0
118                                    Interstellar           8.0
119                         Guardians of the Galaxy           7.9
125                              The Imitation Game           8.0
128                                       Gone Girl           7.9
138                        The Grand Budapest Hotel           7.9
370                                       Inception           7.9
599                                 The Dark Knight           8.1
872                                     The Pianist           7.9
1081  The Lord of the Rings: The Return of the King           7.9
1183                        The Wolf of Wall Street           7.9
1191                               12 Years a Slave           7.9
1800                                        Memento           7.9


# 27. Which actors are most often filmed together? 


In [36]:
#variant1
couple_list=[] 

for actors_list in data.cast_list:
    for couple in combinations(actors_list, 2):
        couple_list.append(' '.join(sorted(couple)))

display(Counter(couple_list).most_common()[0:1])

answers['27'] = 'Daniel Radcliffe Rupert Grint'

[('Daniel Radcliffe Rupert Grint', 8)]

Variant 2


In [37]:
#variant2
data['pairs'] = data['cast_list'].apply(lambda s: list(combinations(s, 2)))
data = data.explode('pairs')
collections.Counter(data['pairs']).most_common(1)

[(('Daniel Radcliffe', 'Rupert Grint'), 8)]

# Submission

In [38]:
# All answers for quastions
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': '110',
 '5': '107',
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': '1478',
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker (tt0267626)',
 '19': '2015',
 '20': '2014',
 '21': '9',
 '22': '450',
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': 'Inside Out, The Dark Knight, 12 Years a Slave',
 '27': 'Daniel Radcliffe Rupert Grint'}